In [8]:
!free -m

              total        used        free      shared  buff/cache   available
Mem:          90614       16364       71779          93        2470       73528
Swap:             0           0           0


In [42]:
!nvidia-smi

Sat Nov 14 18:11:41 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:05.0 Off |                  Off |
| N/A   49C    P0   135W / 300W |  25152MiB / 32480MiB |     83%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
import numpy as np
import pandas as pd
import cv2 
import os
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import skimage.io

In [5]:
!pwd

/home/workspace/user-workspace/OCR/Madfalcon


In [4]:
groups_folder_path = '/home/workspace/data/.train/.task157/test/'
datasetcsv = pd.read_csv('/home/workspace/data/.train/.task157/test/test.csv',sep=',')
datasetcsv

,name,ans
0,01_[00_01_01].tif,NaN
1,01_[00_07_01].tif,NaN
2,01_[02_04_01].tif,NaN
3,01_[02_07_01].tif,NaN
4,01_[03_06_01].tif,NaN
...,...,...
7995,80_[56_14_01].tif,NaN
7996,80_[57_00_01].tif,NaN
7997,80_[57_01_01].tif,NaN
7998,80_[57_13_01].tif,NaN


## 학습파일 npz 저장

In [5]:
groups_folder_path = '/home/workspace/data/.train/.task157/train/'
datasetcsv = pd.read_csv('/home/workspace/data/.train/.task157/train/train.csv',sep=',')
datasetcsv

,name,ans
0,01_[00_03_01].tif,환자
1,01_[00_04_01].tif,성명
2,01_[01_10_01].tif,[
3,01_[01_11_01].tif,]야간
4,01_[01_12_01].tif,[
...,...,...
12833,80_[57_14_01].tif,(16W-60)
12834,80_[58_01_01].tif,수 납 일 자
12835,80_[59_01_01].tif,현금영수증 (
12836,80_[59_04_01].tif,)


In [6]:
# # ans 가 0 인 경우 제거
# idx_ans = datasetcsv[datasetcsv['ans']== '0'].index
# datasetcsv1 = datasetcsv.drop(idx_ans)

# ans 항목 NaN값인 애들 제거
index_nan = datasetcsv[datasetcsv['ans'].isnull() == True].index
datasetcsv.drop(index_nan, inplace=True)

In [7]:
# 전체 값 리스트
dataset_id_list = datasetcsv[['name', 'ans']]
dataset_id_list = dataset_id_list.values.tolist()

In [113]:
# # 가장큰 width 와 height 를 구하기

# ans_weight_list = set()
# for i in tqdm(datasetcsv.index):
#     img_name = datasetcsv['name'][i]
#     ans = datasetcsv['ans'][i]
#     img = cv2.imread(groups_folder_path+img_name)
#     width = img.shape[1]
#     ans_weight_list.add((len(ans),width))
# #     img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
# #     plt.imshow(img)
# #     plt.show()
# print(ans_weight_list)
# # print(max_shape[0]//12838 ,max_shape[1]//12838 )



{(4, 119), (1, 113), (10, 427), (5, 83), (9, 319), (3, 159), (8, 274), (9, 355), (2, 50), (7, 165), (5, 204), (6, 169), (10, 319), (4, 96), (8, 395), (7, 250), (1, 90), (6, 205), (10, 175), (15, 519), (3, 136), (7, 286), (18, 524), (9, 332), (3, 172), (19, 525), (7, 142), (13, 451), (5, 181), (9, 188), (13, 487), (1, 67), (11, 377), (5, 217), (6, 182), (12, 342), (7, 263), (1, 103), (10, 417), (5, 73), (3, 149), (1, 188), (4, 230), (8, 349), (13, 464), (11, 354), (5, 194), (6, 159), (9, 201), (14, 508), (7, 240), (8, 205), (1, 80), (3, 126), (8, 241), (19, 250), (6, 280), (4, 207), (9, 322), (6, 316), (5, 171), (8, 362), (1, 57), (12, 332), (11, 367), (6, 172), (4, 99), (3, 103), (7, 253), (5, 292), (4, 184), (9, 299), (3, 139), (14, 377), (12, 453), (6, 293), (4, 220), (8, 339), (11, 344), (1, 34), (5, 184), (6, 149), (14, 498), (2, 115), (7, 230), (1, 70), (12, 345), (9, 276), (3, 116), (18, 504), (12, 430), (6, 270), (4, 197), (7, 351), (12, 466), (5, 161), (6, 126), (1, 47), (12, 

In [8]:
# max_shape = (54,540)
# rows = 10
# cols = 10
# fig, axes = plt.subplots(rows, cols, figsize=(20, 20))
# # fig.tight_layout()
# i = 1
 
# for index in tqdm(range(100)):
#     img_path = dataset_id_list[index][0]
#     img_name = dataset_id_list[index][1]
#     if len(img_name) == 1:
#         img = cv2.imread(groups_folder_path + img_path)
#         if img.shape[1] != 540: #이미지 확대 할 경우 바이큐빅 또는 쌍 선형 보간법 사용
#             img = cv2.resize(img, dsize=max_shape, interpolation=cv2.INTER_LINEAR)
# #             img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
# #             img = cv2.resize(img, dsize=max_shape, interpolation=cv2.INTER_AREA)
#         ax = fig.add_subplot(rows, cols, i)
#         ax.imshow(img, aspect="auto")
#         ax.set_xlabel(img_name)
#         i += 1
# #     median = cv2.medianBlur(img, 3)
# #     img_blur = cv2.normalize(img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
# #     ax = fig.add_subplot(rows, cols, i)
# #     ax.imshow(img_blur, aspect="auto")
# #     ax.set_xlabel(img_name)
# #     i += 1
# plt.show()

In [9]:
# 모든 이미지 사이즈의 평균 사이즈
mean_size = (54, 130)
# 모든 이미지 사이즈의 최대 사이즈
max_size = (540,54)

# 사진은 X 값을 Y에 저장
X = []
Y = []

for name, ans  in tqdm(dataset_id_list):
    image_dir = groups_folder_path + name
    img = cv2.imread(image_dir)
    if img is not None:  
        # 이미지 처리 
#         img = tile(img) 
        img = cv2.resize(img, dsize=max_size, interpolation=cv2.INTER_AREA)
        img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
#         img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        X.append(img)        
        img_ans = np.array(ans)
        img_ans = img_ans.tolist()
#         print(img_ans)
        Y.append(img_ans)


In [10]:
# X, Y1 array
X = np.array(X, dtype = np.float32)
Y = np.array(Y)

In [11]:
!pwd

/home/workspace/user-workspace/OCR/Madfalcon


In [12]:
# 데이터 split 해서 저장
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, shuffle=True)


np.savez("/home/workspace/user-workspace/OCR/Madfalcon/train_remove_nan.npz", xtrain = X_train, anstrain = Y_train)
np.savez("/home/workspace/user-workspace/OCR/Madfalcon/test_remove_nan.npz", xtest = X_test, anstest = Y_test)